Introduction:

#Introduction to be written here

Import libraries

In [ ]:
import numpy as np
import pandas as pd

WorldCupPlayers data

Reading WorldCupPlayers data

In [ ]:
worldcup_players = pd.read_csv('../data/WorldCupPlayers.csv')

Exploring WorldCupPlayers data

In [ ]:
worldcup_players.head(10)

In [ ]:
worldcup_players.info()

No null values in any columns except for Position and Event

In [ ]:
worldcup_players.dtypes

In [ ]:
worldcup_players.describe()

In [ ]:
worldcup_players.describe(include='object')

If Position cell is null, that means that player's position is unknown, hence it will be filled with 'Unknown'

In [ ]:
worldcup_players['Position'] = worldcup_players['Position'].fillna('Unknown')

If Event cell is null, that means that player didn't record any event in this match, hence it will be filled with empty string

In [ ]:
worldcup_players['Event'] = worldcup_players['Event'].fillna('')

Checking null cells count in table

In [ ]:
worldcup_players.info()

Checking unique values in RoundID

In [ ]:
worldcup_players['RoundID'].unique()

Checking unique values in MatchID

In [ ]:
worldcup_players['MatchID'].unique()

Updating data types for numerical columns, as they are all integr values, and doesn't need to be float. Also optimizing bits  in the data types depending on the needed number of bits, to optimize memory utilisation

In [ ]:
worldcup_players['RoundID'] = worldcup_players['RoundID'].astype(np.uint32)
worldcup_players['MatchID'] = worldcup_players['MatchID'].astype(np.uint32)
worldcup_players['Shirt Number'] = worldcup_players['Shirt Number'].astype(np.uint8)

Cheking dtype of columns after modification